In [1]:
import random

from dataclasses import dataclass

import numpy as np

@dataclass
class Route:
    name: str
    km: int
    trip_every: int

@dataclass
class Activity:
    name: str
    everykm: int

class Schedule:
    KM_MIN = 100
    KM_MAX = 1000
    TRIP_EVERY = [15, 30, 60, 120, 240, 480]
    ACT_EVERY = [2000, 4000, 8000, 16000]
    MORNING = 6
    EVENING = 23
    def __init__(self) -> None:
        self.routes = []
        self.activities = []
        self.trips = []
        
    def add_route(self) -> Route:
        name = 'route%d'%len(self.routes)
        km = random.randrange(self.KM_MIN, self.KM_MAX)
        trip_every = random.choice(self.TRIP_EVERY)
        route = Route(name, km, trip_every)
        self.routes.append(route)
        return route
    
    def add_activity(self) -> Activity:
        name = 'activity%d'%len(self.activities)
        everykm = random.choice(self.ACT_EVERY)
        act = Activity(name, everykm)
        self.activities.append(act)
        return act

    def randomize(self, nr_routes: int, nr_activites: int) -> None:
        for _ in range(nr_routes):
            self.add_route()
        for _ in range(nr_activites):
            self.add_activity()
        self.generate_trips()
    
    def generate_trips(self) -> None:
        for route in self.routes:
            now = self.MORNING * 60
            while now < self.EVENING * 60:
                self.trips.append((now, route.km))
                now += route.trip_every
        self.trips.sort()
        
    def get_arrays(self):
        return (np.array([trip[0] for trip in self.trips], dtype=np.uint64),
                np.array([trip[1] for trip in self.trips], dtype=np.uint64),
                np.array([act.everykm for act in self.activities], dtype=np.uint64))

In [27]:
import math

from numba import jit, jitclass, types
from numba import int64, boolean, float64
from numba.typed import Dict

from collections import Counter

floatarr = float64[:]
spec = {
    'departures': int64[:],
    'distances': int64[:],
    'activities': int64[:],
    'taken': int64[:],
    'km_since': int64[:],
    'now': int64,
    'idx': int64,
    'nrtaken': int64,
    'used_trains': int64,
    'activity_time': int64,
    'p_act': float64[:],
    
    'mcts_explore': float64,
    'mcts_skipprob': float64,
    'mcts_noforce_fac': float64,
    'mcts_activityexp': float64,
    'mcts_triphash': int64,
    'mcts_tripints': int64[:],
    'mcts_visited': types.DictType(int64, int64),
    'mcts_outcomes': types.DictType(int64, float64),
    'mcts_policies': types.DictType(int64, floatarr),
}

@jitclass(spec)
class Simulator:
    def __init__(self, departures, distances, activities, mcts_tripints,
                 mcts_explore, mcts_skipprob, mcts_noforce_fac, mcts_activityexp) -> None:
        self.departures = departures
        self.distances = distances
        self.activities = activities
        
        self.mcts_explore = mcts_explore
        self.mcts_skipprob = mcts_skipprob
        self.mcts_noforce_fac = mcts_noforce_fac
        self.mcts_activityexp = mcts_activityexp
        self.mcts_visited = Dict.empty(
            key_type=int64,
            value_type=int64
        )
        self.mcts_outcomes = Dict.empty(
            key_type=int64,
            value_type=float64
        )
        self.mcts_policies = Dict.empty(
            key_type=int64,
            value_type=floatarr
        )
        self.mcts_tripints = mcts_tripints

        self.reset()
    
    @property
    def done(self) -> bool:
        return self.nrtaken == self.departures.shape[0]
    
    @property
    def score(self) -> float:
        used = self.used_trains
        if self.idx == 0:
            used -= 1
        return used + self.activity_time / 10000
    
    @property
    def mcts_score(self) -> float:
        trips = self.departures.shape[0]
        return (trips - self.score) / trips
    
    def reset(self) -> None:
        self.taken = np.zeros_like(self.departures, dtype=np.uint64)
        self.km_since = np.zeros_like(self.activities, dtype=np.uint64)
        self.p_act = np.ones(self.activities.shape[0] + 2, dtype=np.float64)
        self.now = 60 * 6
        self.idx = 0
        self.nrtaken = 0
        self.used_trains = 1
        self.activity_time = 0
        self.mcts_triphash = 0
        
    def reset_mcts(self) -> None:
        self.mcts_visited.clear()
        self.mcts_outcomes.clear()
        self.mcts_policies.clear()

        
    def legal_trip(self, t_i: int) -> int:
        dist = self.distances[t_i]
        for i in range(self.activities.shape[0]):
            if self.km_since[i] + dist > self.activities[i]:
                return i
        return -1
    
    def take_activity(self, a_i: int) -> None:
        self.km_since[a_i] = 0
        t = self.activities[a_i] // 100
        self.now += t
        self.activity_time += t
        
    def take_trip(self, t_i) -> None:
        self.taken[t_i] = 1
        self.km_since += self.distances[t_i]
        self.now = self.departures[t_i] + self.distances[t_i]
        self.nrtaken += 1
        for i in range(t_i + 1, self.departures.shape[0]):
            if self.departures[i] >= self.now:
                self.idx = i
                return
        self.idx = 0
        self.used_trains += 1
        self.mcts_triphash ^= self.mcts_tripints[t_i]
        
    def next_available(self) -> int:
        for i1 in range(self.idx, self.departures.shape[0]):
            if not self.taken[i1]:
                return i1
            
        for i2 in range(0, self.idx):
            if not self.taken[i2]:
                return i2
            
        raise ValueError
        
    ######################  SOLVERS  ###########################
    
    ######################  GREEDY   ###########################
    def greedy(self) -> None:
        while not self.done:
            i = self.next_available()
            block = self.legal_trip(i)
            if block != -1:
                self.take_activity(block)
                continue
            self.take_trip(i)
    
    ####################  MONTE CARLO  #########################
    def monte_carlo(self) -> None:
        while not self.done:
            i = self.next_available()
                
            actx = np.random.randint(0, self.activities.shape[0] + 2)
            self.take_action(actx)
        return self.score
                
    def policy(self, i) -> None:
        dist = self.distances[i]
        self.p_act.fill(0)
        forced = False
        for i in range(self.activities.shape[0]):
            if self.km_since[i] + dist > self.activities[i]:
                self.p_act[i] = 1
                forced = True

        if forced:
            self.p_act[-1] = self.mcts_skipprob
            self.p_act /= self.p_act.sum()
            return
    
        for i in range(self.activities.shape[0]):
            self.p_act[i] = (self.km_since[i] + dist) / self.activities[i]
            self.p_act[i] ** self.mcts_activityexp

        self.p_act[-2] = 1
        self.p_act[-1] = self.mcts_skipprob/self.mcts_noforce_fac
        self.p_act /= self.p_act.sum()

        
    def mc_n(self, n: int) -> None:
        bestscore = 1000000
        for _ in range(n):
            self.reset()
            self.monte_carlo()
            score = self.score
            if bestscore > score:
                bestscore = score
        return bestscore
    
    ######################  MCTS   ###########################
    def mcts_n(self, n):
        best = 0
        for _ in range(n):
            self.reset()
            self.mcts()
            if self.mcts_score > best:
                #print(self.score, self.mcts_score)
                best = self.mcts_score
        return best
    
    def mcts_do(self, start_actions, improve_limit):
        best = 0
        bestact = 0
        no_improve = 0
        while no_improve < improve_limit:
            self.reset()
            for act in start_actions:
                self.take_action(act)
            score, act = self.mcts()
            no_improve += 1
            if score > best:
                # print('  ', self.score, score, act)
                best = score
                bestact = act
                no_improve = 0

        return bestact, best
        
    def mcts_do_all(self, improve_limit):
        actions = [2]
        actions.pop()
        trips = 0
        bestscore = 0
        while trips < self.departures.shape[0]:
            self.reset_mcts()
            act, score = self.mcts_do(actions, 1 + int(improve_limit * trips / self.departures.shape[0]))
            actions.append(act)
            
            if act == self.activities.shape[0]:
                trips += 1
            if act == self.activities.shape[0] + 1:
                # print('skip!')
                pass
            if score > bestscore:
                bestscore = score
                # print(score, trips, self.departures.shape[0])
        return actions, bestscore
        
    def real_hash(self) -> int:
        hsh = self.mcts_triphash + self.used_trains + self.activity_time
        hsh ^= (self.km_since * self.mcts_tripints[:self.km_since.shape[0]]).sum()
        return hsh
    
    def mcts(self) -> None:
        maxdepth = self.departures.shape[0] * 3
        hsh = np.zeros(maxdepth, dtype=np.int64)
        act = np.zeros(maxdepth, dtype=np.int64)
        depth = 0
        firstact = 0
        while not self.done:
            if depth >= maxdepth:
                break
            
            hsh[depth] = self.real_hash()
            i = self.next_available()
            if hsh[depth] not in self.mcts_policies:
                self.policy(i)
                self.mcts_policies[hsh[depth]] = self.p_act.copy()

            act[depth] = self.highest_ucb(hsh[depth])
            if act[depth] == self.activities.shape[0]:
                self.take_trip(i)
            else:
                self.take_activity(act[depth])
            depth += 1
        
        if self.done:
            score = self.mcts_score
        else:
            score = 0
        for d2 in range(depth):
            hsha = hsh[d2] ^ self.mcts_tripints[act[d2]]
            self.mcts_outcomes[hsha] = (self.mcts_visited.get(hsha, 0) * self.mcts_outcomes.get(hsha, 0) + score) / (
                                        self.mcts_visited.get(hsha, 0) + 1)
            self.mcts_visited[hsha] = self.mcts_visited.get(hsha, 0) + 1
            self.mcts_visited[hsh[d2]] = self.mcts_visited.get(hsh[d2], 0) + 1

        return score, act[0]
    
    def highest_ucb(self, hsh: int) -> int:
        "Calculate action in state hsh with highest UCB1"
        cur_best = -np.inf
        best_act = -1
        
        for act, aval in enumerate(self.mcts_policies[hsh]):
            hsha = hsh ^ self.mcts_tripints[act]
            u = self.mcts_outcomes.get(hsha, 0) + self.mcts_explore * aval * math.sqrt(
                self.mcts_visited.get(hsh, 0) + 1) / (1 + self.mcts_visited.get(hsha, 0))

            if u > cur_best:
                cur_best = u
                best_act = act

        if best_act is -1:
            raise ValueError
        return best_act

    def take_action(self, act):
        if act == self.activities.shape[0] + 1:
            i = self.next_available()
            self.idx = i + 1
        elif act == self.activities.shape[0]:
            self.take_trip(self.next_available())
        else:
            self.take_activity(act)
    
    def action_probabilities(self):
        rets = []
        tot = 0
        hsh = self.real_hash()
        for act, aval in enumerate(self.mcts_policies[hsh]):
            hsha = hsh ^ self.mcts_tripints[act]
            rets.append((act, self.mcts_visited.get(hsha, 0)))
            tot += rets[-1][1]
    
        return [(x[0], x[1] / tot) for x in rets]

### Todo
- kod och körningar
- bilder
- Testexport

### Stretch
- Github
- Länkar

# Komma igång med AI och ML
*(men mest ett föredrag om tågplanering!)*

# Resurser

- De två arketyperna
- Python! (Anaconda)
- Jupyter Notebook!
- Sklearn!
- Onlinekurser, t.ex [https://fast.ai]()
- Pytorch vs Tensorflow
- Följ Towards Data Science och Machine Learning Mastery och [https://tenfifty.io/blog/]()
- Kom på vår utbildning!

# Planering och Reinforcement Learning

Vi planerar en process med:
- En fast uppsättning handlingar
- En målfunktion
- En simulator
- Eventuellt stokastisk

## Tillämpningsområden
 - Robotar
 - Hissplanering
 - Optimera trafikljus
 - Systemkonfiguration (kluster, webservrar, etc)
 - Optimera kemiska reaktioner
 - Rekommendationsmotorer
 - Logistikplanering
 - Resursplanering
 - Prissättning
 - Brädspel
 - Datorspel

# Planera Tåg!

![Tåg](https://upload.wikimedia.org/wikipedia/commons/7/77/Old-Train-Engine.JPG)

## Problemställning
- En tågoperatör ska köra tåg för en satt tidtabell
- Tidtabellen innehåller resor av olika längder till olika platser
- Var X:e km ska vissa aktiviteter utföras
 - Underhåll
 - Toalett-tömning
 - Städning
 - Tankning
 - Påfyllning av varor
- Planera vilket tåg som ska ta vilken sträcka för att komma undan med så få tåg som möjligt (och så få aktivitetsminuter som möjligt)
- Olika aktiviter tar olika lång tid

## Riktiga problemet
- Aktiviteter kan bara utföras på en viss tid
- Ett visst antal aktiviteter kan bara utföras samtidigt
- De utförs på olika platser
- Stationernas utformning bestämmer vilka tåg som får ta vilka resor
- Det finns diverse ställtider för olika handlingar
- Olika veckodagar har olika regler och tidtabell

## Generera syntetiska problem

In [14]:
sched = Schedule()
sched.randomize(5, 2)
sched.routes

[Route(name='route0', km=904, trip_every=120),
 Route(name='route1', km=968, trip_every=120),
 Route(name='route2', km=745, trip_every=30),
 Route(name='route3', km=825, trip_every=60),
 Route(name='route4', km=803, trip_every=30)]

In [15]:
sched.activities

[Activity(name='activity0', everykm=4000),
 Activity(name='activity1', everykm=8000)]

In [17]:
sched.trips[:15]

[(360, 745),
 (360, 803),
 (360, 825),
 (360, 904),
 (360, 968),
 (390, 745),
 (390, 803),
 (420, 745),
 (420, 803),
 (420, 825),
 (450, 745),
 (450, 803),
 (480, 745),
 (480, 803),
 (480, 825)]

# Lösningar
![](https://upload.wikimedia.org/wikipedia/commons/3/33/Balanced_tree.png)

## Simulering
- Simulera endast ett tåg
- För varje dygn, behövs i praktiken ett nytt tåg
- Tågen växlar igenom alla omlopp, så tåg1 tar först dag1 och sedan dag2, dag3, etc.
- Alla tåg kör samma sträcka, fast med olika "fas"

# Slump
- Planering helt utan kunskap
- Slumpa alla handlingar
- Kör många gånger
- Gör inte såhär!

In [28]:
arrs = sched.get_arrays()
tripints = np.frombuffer(np.random.bytes(8 * arrs[0].shape[0]), dtype=np.int64)
        
sim = Simulator(*arrs, tripints, 100, 0.5, 10, 1.5)

# Monte Carlo Tree Search (MCTS)
![](https://www.chessprogramming.org/images/5/50/UCTFormula.jpg)
![](https://upload.wikimedia.org/wikipedia/commons/3/33/Balanced_tree.png)

# Handkoda

- Sätt egna regler, givet tillstånd
- Kanske vanligast i praktiken?
- Finns bättre sätt!

Ta alltid nästa avgång tills en aktivitet måste utföras - utför aktiviteten.

In [33]:
sim.greedy()
sim.score

83.588

# Predictor + UCT (PUCT)
![](https://www.chessprogramming.org/images/5/50/UCTFormula.jpg)
![](https://upload.wikimedia.org/wikipedia/commons/3/33/Balanced_tree.png)

In [40]:
sim.reset_mcts()
sim.mcts_do_all(10000)
sim.score

82.176

In [34]:
sim.mcts_do_all(1000)
sim.score

82.168

# Deep Minds PUCT-variant
![](https://www.chessprogramming.org/images/5/50/UCTFormula.jpg)
![](https://upload.wikimedia.org/wikipedia/commons/3/33/Balanced_tree.png)

# Bayesiansk optimering
![](https://cdn-images-1.medium.com/max/1120/1*PhKGj_bZlND8IEfII426wA.png)

# Sammanfattning
Maskininlärning är inte bara deep learning!

## Mjukvara
- Jupyter Notebook
- numpy
- Numba
- Dragonfly
- Egenkodad MCTS (men det finns generella paket)

## Kod
- Githublänk

![](http://slides.tenfifty.io/imgs/Tenfifty%202.svg)

In [3]:
import dragonfly

In [4]:
import sys
import time
params = [(random.randrange(5, 20), random.randrange(2, 4)) for _ in range(5)]

scheds = []
for trip, aa in params:
    sched = Schedule()
    sched.randomize(trip, aa)
    scheds.append(sched)

def hyperscore(z, x):
    print(z)
    print(x)
    mcts_explore, mcts_skipprob, mcts_noforce_fac, mcts_activityexp = x
    
    tot = 0
    for sched in scheds:
        arrs = sched.get_arrays()
        tripints = np.frombuffer(np.random.bytes(8 * arrs[0].shape[0]), dtype=np.int64)
        
        sim = Simulator(*arrs, tripints, mcts_explore, mcts_skipprob, mcts_noforce_fac, mcts_activityexp)
        _, score = sim.mcts_do_all(z[0])
        tot += score
    print(time.ctime(), tot)
    sys.stdout.flush()
    return tot
        

In [6]:
maxscore, bestargs, history = dragonfly.maximize_multifidelity_function(
    hyperscore, [[100, 500]], [[1, 100], [0.01, 0.5], [1, 10], [1, 2]], 500, lambda x: 5*x/10, max_capital=3600, capital_type='realtime')

Optimisation with mfbo-boca(ei-ucb-ttei-ts-add_ucb) using capital 3600.0 (realtime)
[495.2892957]
[70.86114355  0.08710384  9.25549141  1.482538  ]
Thu Mar 28 11:55:01 2019 2.367361871502644
using point
Capital spent on initialisation: 101.1240(0.0281).
Legend: <iteration_number> (<num_successful_queries>) cap=<fraction_of_capital_spent>:: curr_max=<current_maximum_value>, f2o=<#queries_at_highest_fidelity>(<#queries_at_highest_fidelity_in_last_20_iterations>), acqs=<num_times_each_acquisition_was_used>
[100.]
[1.00251486 0.49999862 1.00022862 1.9999746 ]
Thu Mar 28 11:55:51 2019 2.367543689684462
skip fidel
[100.]
[11.42628723  0.02172425  8.29841861  1.84076883]
Thu Mar 28 11:56:15 2019 2.3672208774360985
skip fidel
[100.]
[1.06790123 0.49988797 1.00205761 1.00022862]
Thu Mar 28 11:56:44 2019 2.3674474329999704
skip fidel
[100.]
[72.5         0.41833333  2.5         1.5       ]
Thu Mar 28 11:57:15 2019 2.3673813052435855
skip fidel
[100.]
[39.5         0.41833333  2.5         1.83333

skip fidel
[100.]
[9.45000000e+01 3.72222222e-02 9.50000000e+00 1.05555556e+00]
Thu Mar 28 12:28:10 2019 2.3672208774360985
skip fidel
[100.]
[6.5        0.47277778 1.5        1.94444444]
Thu Mar 28 12:28:33 2019 2.3678134863623472
skip fidel
[100.]
[1.00251486 0.49999585 1.00007621 1.99999153]
Thu Mar 28 12:28:54 2019 2.3677008516954103
skip fidel
[100.]
[1.00251486 0.49999585 1.00007621 1.9999746 ]
Thu Mar 28 12:29:18 2019 2.3677008516954103
skip fidel
[100.]
[1.00251486 0.49999585 1.00007621 1.99999153]
Thu Mar 28 12:29:46 2019 2.367543689684462
skip fidel
[100.]
[2.83333333 0.27314815 1.5        1.94444444]
Thu Mar 28 12:30:12 2019 2.36737372292626
skip fidel
[100.]
[1.00754458 0.49996266 1.00205761 1.99992379]
Thu Mar 28 12:30:35 2019 2.367737686085186
skip fidel
[100.]
[1.00083829 0.49999585 1.00007621 1.9999746 ]
Thu Mar 28 12:30:58 2019 2.367693129687688
skip fidel
[100.]
[2.27206131 0.49797608 1.75888671 1.85707352]
Thu Mar 28 12:31:23 2019 2.3677099812625184
skip fidel
[100.]

In [5]:
dragonfly.maximize_multifidelity_function(
    hyperscore, [[100, 500]], [[1, 100], [0.01, 0.5], [1, 10], [1, 2]], 
    500, lambda x: 5*x/10, max_capital=60*60*6, capital_type='return_value')

Optimisation with mfbo-boca(ei-ucb-ttei-ts-add_ucb) using capital 21600.0 (return_value)
[266.57984907]
[63.2626126   0.49203279  8.17688899  1.97704345]
0.459873417721519 1 553
Thu Mar 28 00:23:35 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 00:23:49 2019 0.8745082300082084
0.3155606936416185 1 173
Thu Mar 28 00:24:01 2019 1.190068923649827
0.41840000000000005 1 100
Thu Mar 28 00:24:03 2019 1.608468923649827
0.420625 1 192
Thu Mar 28 00:24:09 2019 2.0290939236498273
[228.02382328]
[41.55919669  0.35053461  6.90135761  1.55107847]
0.459873417721519 1 553
Thu Mar 28 00:25:43 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 00:25:53 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 11 173
0.3156069364161849 15 173
0.3156300578034682 25 173
0.3156531791907515 50 173
0.31567630057803464 58 173
0.3156994219653179 65 173
0.3157225433526012 108 173
0.31574566473988436 112 173
0.31576878612716763 120 173
0.3157919075144509 125 173
0.3158150289017341 1

0.420625 1 192
Thu Mar 28 00:55:02 2019 2.0293251375226595
[403.08829938]
[1.24114107 0.12941333 8.59879347 1.45882686]
0.459873417721519 1 553
Thu Mar 28 00:57:00 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 00:57:18 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 27 173
0.3156069364161849 58 173
0.3156300578034682 65 173
0.3156531791907515 108 173
0.31567630057803464 112 173
0.3156994219653179 125 173
Thu Mar 28 00:57:34 2019 1.1902076519735263
0.41840000000000005 1 100
Thu Mar 28 00:57:36 2019 1.6086076519735264
0.420625 1 192
Thu Mar 28 00:57:44 2019 2.029232651973526
[173.8822229]
[8.51071481 0.13153187 4.01903278 1.99116832]
0.459873417721519 1 553
Thu Mar 28 00:58:40 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 00:58:48 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 58 173
0.3156069364161849 65 173
0.3156300578034682 84 173
0.3156531791907515 96 173
0.31567630057803464 108 173
0.3156994219653179 112 173
0.315

0.420625 1 192
Thu Mar 28 01:21:12 2019 2.0294176230717924
[467.79627435]
[85.270151    0.22653472  8.15465769  1.51308147]
0.459873417721519 1 553
Thu Mar 28 01:24:12 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 01:24:35 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 11 173
0.3156069364161849 15 173
0.3156300578034682 25 173
0.3156531791907515 50 173
0.31567630057803464 58 173
0.3156994219653179 65 173
0.3157225433526012 108 173
0.31574566473988436 112 173
0.31576878612716763 120 173
0.3157919075144509 125 173
0.3158150289017341 132 173
0.31583815028901735 170 173
Thu Mar 28 01:24:54 2019 1.1903463802972258
0.41840000000000005 1 100
Thu Mar 28 01:24:56 2019 1.608746380297226
0.420625 1 192
Thu Mar 28 01:25:07 2019 2.029371380297226
[169.64196066]
[43.1834829   0.42584223  3.60396938  1.06554028]
0.459873417721519 1 553
Thu Mar 28 01:26:19 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 01:26:27 2019 0.8745082300082084
0.3155606936416185 1

0.4146348122866894 1 293
Thu Mar 28 01:54:19 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 15 173
0.3156069364161849 25 173
0.3156300578034682 50 173
0.3156531791907515 58 173
0.31567630057803464 65 173
0.3156994219653179 66 173
0.3157225433526012 108 173
0.3157919075144509 112 173
0.3158150289017341 114 173
0.31583815028901735 125 173
0.3158612716763006 137 173
0.3158843930635838 158 173
0.31590751445086707 168 173
Thu Mar 28 01:54:28 2019 1.1904157444590755
0.41840000000000005 1 100
Thu Mar 28 01:54:29 2019 1.6088157444590756
0.420625 1 192
Thu Mar 28 01:54:35 2019 2.029440744459076
[462.83615669]
[39.93599737  0.29358131  2.45944597  1.42892819]
0.459873417721519 1 553
Thu Mar 28 01:58:11 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 01:58:35 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 8 173
0.3156069364161849 15 173
0.3156300578034682 25 173
0.3156531791907515 50 173
0.31567630057803464 58 173
0.3156994219653179 64 173
0.

0.3155606936416185 1 173
Thu Mar 28 02:24:03 2019 1.190068923649827
0.41840000000000005 1 100
Thu Mar 28 02:24:04 2019 1.608468923649827
0.420625 1 192
Thu Mar 28 02:24:08 2019 2.0290939236498273
[460.19159818]
[50.28044526  0.40518365  6.38013186  1.9651614 ]
0.459873417721519 1 553
Thu Mar 28 02:27:59 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 02:28:22 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 8 173
0.3156069364161849 15 173
0.3156300578034682 25 173
0.3156531791907515 50 173
0.31567630057803464 58 173
0.3156994219653179 64 173
0.3157225433526012 108 173
0.31574566473988436 112 173
0.31576878612716763 120 173
0.3157919075144509 125 173
0.3158150289017341 132 173
0.31583815028901735 170 173
Thu Mar 28 02:28:41 2019 1.1903463802972258
0.41840000000000005 1 100
Thu Mar 28 02:28:43 2019 1.608746380297226
0.420625 1 192
Thu Mar 28 02:28:55 2019 2.029371380297226
[500.]
[5.60771958 0.12792958 2.54099758 1.72965215]
0.459873417721519 1 553
Thu Mar 

...

0.3164393063583815 151 173
0.31646242774566474 153 173
0.31647398843930635 157 173
0.31648554913294796 160 173
0.31649710982658963 163 173
0.31650867052023124 166 173
0.3165202312138728 167 173
0.31653179190751446 169 173
0.31654335260115607 170 173
0.31656647398843935 172 173
Thu Mar 28 06:24:02 2019 1.1910747039966478
0.41840000000000005 1 100
Thu Mar 28 06:24:05 2019 1.609474703996648
0.420625 1 192
Thu Mar 28 06:24:35 2019 2.030099703996648
[436.77088828]
[33.22256566  0.22802924  3.01423398  1.5352467 ]
0.459873417721519 1 553
Thu Mar 28 06:27:32 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 06:27:55 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 11 173
0.3156069364161849 15 173
0.3156300578034682 25 173
0.3156531791907515 50 173
0.31567630057803464 58 173
0.3156994219653179 65 173
0.3157225433526012 108 173
0.31574566473988436 112 173
0.31576878612716763 120 173
0.3157919075144509 125 173
0.3158150289017341 132 173
0.3158843930635838 153 173
0.3

[260.12595245]
[60.49555761  0.47635146  1.1348119   1.81770165]
0.459873417721519 1 553
Thu Mar 28 06:59:11 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 06:59:24 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 8 173
0.3156069364161849 15 173
0.3156300578034682 16 173
0.3156416184971098 29 173
0.3156647398843931 37 173
0.3156878612716763 50 173
0.3156994219653179 52 173
0.3157109826589595 53 173
0.315757225433526 54 173
0.31578034682080924 56 173
0.31580346820809246 57 173
0.3158150289017341 59 173
0.31582658959537574 60 173
0.31584971098265896 64 173
0.3158612716763006 66 173
0.3158728323699422 69 173
0.31589595375722546 71 173
0.31590751445086707 73 173
0.3159190751445087 79 173
0.31593063583815034 82 173
0.3159537572254335 84 173
0.31600000000000006 96 173
0.3160231213872832 109 173
0.3160462427745665 111 173
0.3161156069364162 113 173
0.3161387283236994 114 173
0.31615028901734105 117 173
0.3161734104046242 120 173
0.3161965317919075 122 173
0.316

0.420625 1 192
Thu Mar 28 07:28:52 2019 2.0294176230717924
[430.33342981]
[74.93068137  0.10099523  7.78275084  1.92979432]
0.459873417721519 1 553
Thu Mar 28 07:31:32 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 07:31:56 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 25 173
0.3156069364161849 50 173
0.3156300578034682 58 173
0.3156531791907515 65 173
0.31567630057803464 108 173
0.3156994219653179 112 173
0.3157225433526012 120 173
0.31574566473988436 125 173
0.31576878612716763 170 173
Thu Mar 28 07:32:16 2019 1.1902770161353762
0.41840000000000005 1 100
Thu Mar 28 07:32:18 2019 1.6086770161353763
0.420625 1 192
Thu Mar 28 07:32:29 2019 2.029302016135376
[500.]
[74.08597649  0.43421758  3.21958592  1.35868039]
0.459873417721519 1 553
Thu Mar 28 07:37:14 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 07:37:39 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 5 173
0.3156069364161849 15 173
0.3156300578034682 16 173
0.31

0.41840000000000005 1 100
Thu Mar 28 08:04:38 2019 1.6090585190255495
0.420625 1 192
Thu Mar 28 08:04:52 2019 2.0296835190255496
[269.96280103]
[20.24213805  0.4209901   2.0118415   1.50294769]
0.459873417721519 1 553
Thu Mar 28 08:07:06 2019 0.459873417721519
0.4146348122866894 1 293
Thu Mar 28 08:07:20 2019 0.8745082300082084
0.3155606936416185 1 173
0.31558381502890176 11 173
0.3156069364161849 15 173
0.3156300578034682 25 173
0.3156531791907515 29 173
0.31567630057803464 50 173
0.3156994219653179 58 173
0.3157225433526012 65 173
0.31574566473988436 108 173
0.31576878612716763 112 173
0.3157919075144509 120 173
0.3158150289017341 125 173
0.3158843930635838 144 173
0.31590751445086707 146 173
0.3159537572254335 166 173
0.3159768786127168 169 173
Thu Mar 28 08:07:29 2019 1.1904851086209252
0.41840000000000005 1 100
Thu Mar 28 08:07:30 2019 1.6088851086209253
0.420625 1 192
Thu Mar 28 08:07:40 2019 2.0295101086209253
[438.33041454]
[52.40849336  0.17142144  9.50122735  1.7579158 ]
0.45

KeyboardInterrupt: 